In [1]:
import re
import matplotlib.pyplot as plt
import numpy as np
import heapq

# Global variables
grid_dimensions = None
bend_penalty = None
via_penalty = None
obstructions = []
nets = {}
grid = []

# Function to parse the input file
def parse_input_file(file_path):
    global grid_dimensions, bend_penalty, via_penalty, obstructions, nets

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the grid dimensions and penalties
    grid_info = list(map(int, lines[0].strip().split(',')))
    grid_dimensions = (grid_info[0], grid_info[1])  # NxM grid
    bend_penalty = grid_info[2]
    via_penalty = grid_info[3]

    # Parse the obstructions
    for line in lines[1:]:
        if line.startswith("OBS"):
            match = re.search(r"\((\d+),\s*(\d+),\s*(\d+)\)", line)
            if match:
                obstructions.append((int(match.group(1)), int(match.group(2)), int(match.group(3))))

    # Parse the nets
    for line in lines[1:]:
        if line.startswith("net"):
            net_name = line.split(' ')[0]
            net_points = re.findall(r"\((\d+),\s*(\d+),\s*(\d+)\)", line)
            nets[net_name] = [(int(layer), int(x), int(y)) for layer, x, y in net_points]

# Function to initialize the grid
def initialize_grid():
    global grid, grid_dimensions, obstructions

    # Extract dimensions
    rows, cols = grid_dimensions

    # Create a 2D list initialized with default cost (0 for simplicity)
    grid = [[{'cost': 0, 'obstruction': False} for _ in range(cols)] for _ in range(rows)]

    # Mark obstructions
    for layer, x, y in obstructions:
        # Use layer-1 indexing to represent each layer separately
        grid[x][y]['obstruction'] = True
        grid[x][y]['cost'] = float('inf')  # Assign a high cost to obstructions

# Function to calculate cost based on penalties
def calculate_cost(current_layer, current_direction, next_layer, next_direction):
    cost = 1  # Base cost for movement
    if current_layer != next_layer:
        cost += via_penalty
    if current_direction != next_direction:
        cost += bend_penalty
    return cost

# Function to find a path for a net
def find_path(start, end):
    global grid, grid_dimensions

    rows, cols = grid_dimensions
    start_layer, start_x, start_y = start
    end_layer, end_x, end_y = end

    # Priority queue for BFS with costs
    pq = []
    heapq.heappush(pq, (0, start, []))  # (cost, current_position, path)

    # Visited set
    visited = set()

    # Directions for movement
    directions = {
        "horizontal": [(0, 1), (0, -1)],  # Right, Left
        "vertical": [(1, 0), (-1, 0)],  # Down, Up
    }

    while pq:
        current_cost, (layer, x, y), path = heapq.heappop(pq)

        # If reached the destination
        if (layer, x, y) == (end_layer, end_x, end_y):
            return path + [(layer, x, y)]

        # Mark the current position as visited
        if (layer, x, y) in visited:
            continue
        visited.add((layer, x, y))

        # Determine preferred direction for the current layer
        preferred_direction = "horizontal" if layer == 0 else "vertical"
        for dx, dy in directions[preferred_direction]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and not grid[nx][ny]['obstruction']:
                new_cost = current_cost + calculate_cost(layer, preferred_direction, layer, preferred_direction)
                heapq.heappush(pq, (new_cost, (layer, nx, ny), path + [(layer, x, y)]))

        # Try switching layers (via)
        new_layer = 1 - layer
        if not grid[x][y]['obstruction']:  # Check for via feasibility
            new_cost = current_cost + calculate_cost(layer, preferred_direction, new_layer, preferred_direction)
            heapq.heappush(pq, (new_cost, (new_layer, x, y), path + [(layer, x, y)]))

    # If no path is found
    return None

# Function to route all nets
def route_nets():
    global nets
    routes = {}

    for net_name, net_points in nets.items():
        print(f"Routing {net_name}...")
        path = []
        for i in range(len(net_points) - 1):
            segment_path = find_path(net_points[i], net_points[i + 1])
            if segment_path is None:
                print(f"Failed to route {net_name}")
                routes[net_name] = None
                break
            path.extend(segment_path)
        else:
            routes[net_name] = path
    return routes

# Function to visualize routed nets
def visualize_routes(routes):
    global grid, grid_dimensions

    rows, cols = grid_dimensions
    grid_matrix = np.zeros((rows, cols), dtype=int)

    for net_name, path in routes.items():
        if path is not None:
            for layer, x, y in path:
                grid_matrix[x][y] += 1  # Increment cell usage for visualization

    fig, ax = plt.subplots(figsize=(25, 25))
    cmap = plt.cm.get_cmap("viridis")
    ax.imshow(grid_matrix, cmap=cmap)

    ax.set_title("Routed Nets Visualization", fontsize=16)
    plt.show()

# Example usage
if __name__ == "_main_":
    input_file = "input.txt"  # Replace with your input file
    parse_input_file(input_file)
    initialize_grid()
    routed_nets = route_nets()
    visualize_routes(routed_nets)

In [18]:
import re
import matplotlib.pyplot as plt
import numpy as np
# Global variables
grid_dimensions = None
bend_penalty = None
via_penalty = None
obstructions = []
nets = {}
grid = []


# Function to parse the input file
def parse_input_file(file_path):
    global grid_dimensions, bend_penalty, via_penalty, obstructions, nets

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the grid dimensions and penalties
    grid_info = list(map(int, lines[0].strip().split(',')))
    grid_dimensions = (grid_info[0], grid_info[1])  # NxM grid
    bend_penalty = grid_info[2]
    via_penalty = grid_info[3]

    # Parse the obstructions
    for line in lines[1:]:
        if line.startswith("OBS"):
            match = re.search(r"\((\d+),\s*(\d+),\s*(\d+)\)", line)
            if match:
                obstructions.append((int(match.group(1)), int(match.group(2)), int(match.group(3))))
    
    # Parse the nets
    for line in lines[1:]:
        if line.startswith("net"):
            net_name = line.split(' ')[0]
            net_points = re.findall(r"\((\d+),\s*(\d+),\s*(\d+)\)", line)
            nets[net_name] = [(int(layer), int(x), int(y)) for layer, x, y in net_points]

# Global grid representation

def initialize_grid():
    global grid, grid_dimensions, obstructions

    # Extract dimensions
    rows, cols = grid_dimensions

    # Create a 3D list for two layers, each initialized with default cost (0 for simplicity)
    grid = [[[{'cost': 0, 'obstruction': False} for _ in range(cols)] for _ in range(rows)] for _ in range(2)]

    # Mark obstructions
    for layer, x, y in obstructions:
        grid[layer][x][y]['obstruction'] = True
        grid[layer][x][y]['cost'] = float('inf')  # Assign a high cost to obstructions



def visualize_3d_grid_matplotlib():
    global grid, grid_dimensions

    # Extract grid dimensions
    rows, cols = grid_dimensions

    # Prepare two grid matrices for the layers
    grid_matrix_layer0 = np.zeros((rows, cols), dtype=int)
    grid_matrix_layer1 = np.zeros((rows, cols), dtype=int)

    for layer in range(2):
        for row in range(rows):
            for col in range(cols):
                if grid[layer][row][col]['obstruction']:
                    if layer == 0:
                        grid_matrix_layer0[row][col] = -1
                    else:
                        grid_matrix_layer1[row][col] = -1

    # Create a side-by-side visualization of the two layers
    fig, axes = plt.subplots(1, 2, figsize=(25, 12))  # One subplot for each layer

    cmap = plt.cm.get_cmap("Greens")  # Color map for free cells
    cmap.set_under(color="red")  # Red for obstructions

    # Plot Layer 0
    axes[0].imshow(grid_matrix_layer0, cmap=cmap, vmin=0, vmax=1)
    axes[0].set_title("Layer M0 (Horizontal)", fontsize=16)
    axes[0].set_xticks(np.arange(-0.5, cols, 1), minor=True)
    axes[0].set_yticks(np.arange(-0.5, rows, 1), minor=True)
    axes[0].grid(which="minor", color="black", linestyle='-', linewidth=0.5)
    axes[0].set_aspect('equal')
    axes[0].set_xticks([])
    axes[0].set_yticks([])

    # Plot Layer 1
    axes[1].imshow(grid_matrix_layer1, cmap=cmap, vmin=0, vmax=1)
    axes[1].set_title("Layer M1 (Vertical)", fontsize=16)
    axes[1].set_xticks(np.arange(-0.5, cols, 1), minor=True)
    axes[1].set_yticks(np.arange(-0.5, rows, 1), minor=True)
    axes[1].grid(which="minor", color="black", linestyle='-', linewidth=0.5)
    axes[1].set_aspect('equal')
    axes[1].set_xticks([])
    axes[1].set_yticks([])

    # Show the plot
    plt.tight_layout()
    plt.show()
def order_nets_by_pins(nets):
    # Sort nets by the number of pins (ascending order)
    return sorted(nets.items(), key=lambda x: len(x[1]))

from collections import deque

def lee_algorithm(grid, start, end, layer, bend_penalty, via_penalty):
    rows, cols = grid_dimensions
    visited = [[[False for _ in range(cols)] for _ in range(rows)] for _ in range(2)]  # Track visited cells
    prev = [[[None for _ in range(cols)] for _ in range(rows)] for _ in range(2)]  # Track predecessors

    # Directions: horizontal/vertical moves
    directions = [
        (0, 1), (0, -1),  # Horizontal
        (1, 0), (-1, 0)   # Vertical
    ]

    # Initialize queue for BFS
    queue = deque([(start[0], start[1], layer)])  # (x, y, layer)
    visited[layer][start[0]][start[1]] = True

    while queue:
        x, y, current_layer = queue.popleft()

        # Check if we reached the target
        if (x, y, current_layer) == (end[0], end[1], layer):
            break

        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Check bounds and obstacles
            if 0 <= nx < rows and 0 <= ny < cols and not visited[current_layer][nx][ny] and not grid[current_layer][nx][ny]['obstruction']:
                visited[current_layer][nx][ny] = True
                prev[current_layer][nx][ny] = (x, y, current_layer)
                queue.append((nx, ny, current_layer))

        # Check for layer change (via)
        if current_layer == 0 and not visited[1][x][y]:  # Change to Layer 1
            if not grid[1][x][y]['obstruction']:
                visited[1][x][y] = True
                prev[1][x][y] = (x, y, current_layer)
                queue.append((x, y, 1))
        elif current_layer == 1 and not visited[0][x][y]:  # Change to Layer 0
            if not grid[0][x][y]['obstruction']:
                visited[0][x][y] = True
                prev[0][x][y] = (x, y, current_layer)
                queue.append((x, y, 0))

    # Reconstruct path
    path = []
    cx, cy, cl = end[0], end[1], layer
    while prev[cl][cx][cy] is not None:
        path.append((cl, cx, cy))
        cx, cy, cl = prev[cl][cx][cy]
    path.append((layer, start[0], start[1]))
    path.reverse()

    return path

def route_nets(grid, nets, bend_penalty, via_penalty):
    # Order nets by heuristic
    ordered_nets = order_nets_by_pins(nets)

    # Store results
    net_paths = {}

    for net_name, pins in ordered_nets:
        net_path = []
        for i in range(len(pins) - 1):
            start_layer, start_x, start_y = pins[i]
            end_layer, end_x, end_y = pins[i + 1]

            # Route between pins using Lee's Algorithm
            path = lee_algorithm(grid, (start_x, start_y), (end_x, end_y), start_layer, bend_penalty, via_penalty)
            net_path.extend(path)

        # Store the path for the net
        net_paths[net_name] = net_path

        # Mark the routed path on the grid to avoid reuse
        for layer, x, y in net_path:
            grid[layer][x][y]['obstruction'] = True

    return net_paths


if __name__ == "__main__":
    input_file = "input.txt"  # Replace with your input file path
    parse_input_file(input_file)  # Parse the input file
    initialize_grid()  # Initialize the 3D grid

    # Route the nets
    routed_nets = route_nets(grid, nets, bend_penalty, via_penalty)

    # Print the routed paths
    for net_name, path in routed_nets.items():
        print(f"{net_name}: {path}")



net1: [(0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 3, 6), (0, 4, 6), (0, 5, 6), (1, 5, 6), (1, 5, 7), (1, 5, 8), (1, 5, 9), (1, 6, 9), (1, 7, 9), (1, 8, 9)]
net2: [(0, 1, 1), (0, 1, 2), (0, 2, 2), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 4, 4), (0, 4, 4), (0, 5, 4), (0, 5, 5), (0, 6, 5), (0, 6, 6)]
net3: [(1, 9, 9), (1, 9, 8), (1, 9, 7), (1, 9, 6), (1, 9, 5), (1, 8, 5), (1, 7, 5), (1, 6, 5), (1, 5, 5), (1, 4, 5), (1, 3, 5), (1, 2, 5), (1, 1, 5), (1, 0, 5), (0, 0, 5), (0, 0, 4), (0, 0, 3), (0, 0, 2), (0, 0, 1), (0, 0, 0), (0, 1, 0), (0, 2, 0)]


In [38]:
from collections import deque

def lee_algorithm_3d(grid, start, end):
    """
    Performs Lee's algorithm with obstruction checking on a 3D grid.
    :param grid: 3D grid where each layer is a 2D representation.
                 grid[layer][row][col]['obstruction'] is True if the cell is obstructed.
    :param start: Starting cell as a tuple (layer, row, col).
    :param end: Ending cell as a tuple (layer, row, col).
    :return: List of cells in the shortest path or None if no path exists.
    """
    if is_obstructed(grid, start) or is_obstructed(grid, end):
        return None  # Start or end is obstructed

    queue = deque()
    visited = set()
    prev = {}

    queue.append(start)
    visited.add(start)

    while queue:
        current = queue.popleft()

        # If we've reached the target
        if current == end:
            return get_shortest_path(prev, start, end)

        # Explore neighboring cells
        for neighbor in get_neighbors_3d(grid, current):
            if neighbor not in visited:
                visited.add(neighbor)
                prev[neighbor] = current
                queue.append(neighbor)

    return None  # No path found


def is_obstructed(grid, cell):
    """
    Checks if a cell in the 3D grid is obstructed.
    :param grid: 3D grid where obstructions are marked as True.
    :param cell: Tuple (layer, row, col) of the cell.
    :return: True if obstructed, False otherwise.
    """
    layer, row, col = cell
    return grid[layer][row][col]['obstruction']


def get_neighbors_3d(grid, node):
    """
    Gets valid neighboring cells for a given node in a 3D grid.
    :param grid: 3D grid where each layer is a 2D representation.
    :param node: Current cell as a tuple (layer, row, col).
    :return: List of valid neighboring cells.
    """
    neighbors = []
    layer, row, col = node

    layers, rows, cols = len(grid), len(grid[0]), len(grid[0][0])

    # Possible 2D directions: up, down, left, right
    directions = [(0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]

    for d_layer, d_row, d_col in directions:
        n_layer, n_row, n_col = layer + d_layer, row + d_row, col + d_col

        # Ensure the neighbor is in bounds and not obstructed
        if 0 <= n_layer < layers and 0 <= n_row < rows and 0 <= n_col < cols and not is_obstructed(grid, (n_layer, n_row, n_col)):
            neighbors.append((n_layer, n_row, n_col))

    # Allow movement to adjacent layers
    for d_layer in [-1, 1]:  # Up or down a layer
        n_layer = layer + d_layer
        if 0 <= n_layer < layers and not is_obstructed(grid, (n_layer, row, col)):
            neighbors.append((n_layer, row, col))

    return neighbors


def get_shortest_path(prev, start, end):
    """
    Backtracks from the end to start to reconstruct the path.
    :param prev: Dictionary of previous nodes.
    :param start: Starting cell.
    :param end: Ending cell.
    :return: List of cells in the shortest path.
    """
    path = []
    current = end

    while current != start:
        path.append(current)
        current = prev[current]

    path.append(start)
    path.reverse()

    return path


# Example usage
# Example grid initialization for 3D
grid = [
    # Layer 0
    [
        [{'obstruction': False} for _ in range(7)]
        for _ in range(5)
    ],
    # Layer 1
    [
        [{'obstruction': False} for _ in range(7)]
        for _ in range(5)
    ]
]

# Example: Add obstructions
grid[0][2][3]['obstruction'] = True  # Add an obstruction in layer 0, row 2, col 3
grid[1][4][5]['obstruction'] = True  # Add an obstruction in layer 1, row 4, col 5

start = (0, 0, 0)  # Start in layer 0, row 0, column 0
end = (1, 4, 6)    # End in layer 1, row 4, column 6

shortest_path = lee_algorithm_3d(grid, start, end)
if shortest_path:
    print("Shortest path:", shortest_path)
else:
    print("No path exists.")


Shortest path: [(0, 0, 0), (0, 1, 0), (0, 2, 0), (0, 3, 0), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (0, 4, 5), (0, 4, 6), (1, 4, 6)]
